# Linear Programming

In [64]:
import numpy as np
import pandas as pd
import random
import warnings
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
import plotly.express as px

In [3]:
# import data
centroid_locations = pd.read_csv(r"C:\Users\dodie\Downloads\spring 2024\trio-capstone\data\CensusTractCentroids.csv")
metro_locations = pd.read_csv(r"C:\Users\dodie\Downloads\spring 2024\trio-capstone\data\MetroLinkStations_REGISTERED.csv")
north_south_locations = pd.read_csv(r"C:\Users\dodie\Downloads\spring 2024\trio-capstone\data\MetroLinkStations_NS.csv")
census_data = data = pd.read_csv(r"C:\Users\dodie\Downloads\spring 2024\trio-capstone\data\B08119_stl_city.csv")

#centroid_locations = pd.read_csv("/Users/dylanmack/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/ESE 499/trio-capstone/data/CensusTractCentroids.csv")
#metro_locations = pd.read_csv("/Users/dylanmack/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/ESE 499/trio-capstone/data/MetroLinkStations_REGISTERED.csv")
#north_south_locations = pd.read_csv("/Users/dylanmack/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/ESE 499/trio-capstone/data/MetroLinkStations_NS.csv")
#census_data = data = pd.read_csv("/Users/dylanmack/Library/CloudStorage/OneDrive-WashingtonUniversityinSt.Louis/ESE 499/trio-capstone/data/B08119_stl_city.csv")

In [4]:
# clean census data

warnings. filterwarnings('ignore')

# Only keep important columns
census_data = census_data[['location','label','estimate','moe']]
census_data['label'] = census_data['label'].str.replace('Estimate', '').str.replace('Total', '').str.replace('!!', '').str.replace(':', '')
census_data['location'] = census_data['location'].str.replace('; St. Louis city; Missouri','')

# Filter out aggregate columns
all_totals = census_data[census_data['label']=='']
trans_totals_mask = ~census_data['label'].astype(str).str.contains('[$]')
trans_totals = census_data.loc[trans_totals_mask][1:]
income_totals = census_data[census_data['label'].str.startswith("$")]
not_to_include = pd.concat([trans_totals, income_totals, all_totals])
merged_df = pd.merge(census_data, not_to_include, how='left', indicator=True)
no_totals = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])[1:]
split_data = no_totals['label'].str.split('$', 1, expand=True)
no_totals[['transportation', 'income']] = split_data
no_totals.drop(columns=['label'], inplace=True)
no_totals['income'] = '$' + no_totals['income']
from_this = no_totals['income'].unique()
to_this = [1, 10000, 15000, 25000, 35000, 50000, 65000, 75000]
no_totals['avg income'] = no_totals['income'].replace(from_this, to_this)
clean_census_data = no_totals

In [6]:
# Group by census tract and mode of transportation
by_tract_trans = clean_census_data.groupby(['location','transportation'])['estimate'].sum()

# Find percentage of people who take public transport in each tract
tract_pops = clean_census_data.groupby('location', as_index = False)['estimate'].sum()['estimate'].tolist()
public_transit_pops = clean_census_data[clean_census_data['transportation']=='Public transportation (excluding taxicab)'].groupby(['location','transportation'], as_index = False)['estimate'].sum()['estimate'].tolist()

pct_public_trans = []
for i in range(len(tract_pops)):
    pct_public_trans.append(public_transit_pops[i]/tract_pops[i])

# Add percentages back in to groupby
by_tract = clean_census_data.groupby('location', as_index = False)['estimate'].sum()
by_tract['public transport count'] = public_transit_pops
by_tract['public transport %'] = pct_public_trans

# Group by census tract and income
by_tract_inc = clean_census_data.groupby(['location','avg income'])['estimate'].sum()

# Find average income of each tract
tract_incomes = (clean_census_data['avg income']*clean_census_data['estimate']).groupby(clean_census_data['location']).sum().tolist()
tract_totals = sum(tract_incomes)
tract_incomes_new = np.array([tract_totals - i for i in tract_incomes])
income_weight = tract_incomes_new / sum(tract_incomes_new)

# Add percentages back in to groupby
by_tract['income weight'] = income_weight

In [7]:
# merge census tract location data with census data

centroid_locations_small = centroid_locations[['NAMELSAD','INTPTLAT','INTPTLON']]
centroid_locations_small = centroid_locations_small.rename(columns = {"NAMELSAD": "location", "INTPTLAT" : "LAT", "INTPTLON" : "LON"})

full_data = by_tract.merge(centroid_locations_small, on='location')
full_data.head()

,location,estimate,public transport count,public transport %,income weight,LAT,LON
0,Census Tract 1011,1039,31,0.029836,0.009653,38.554718,-90.273620
1,Census Tract 1012,1921,11,0.005726,0.009570,38.561824,-90.274863
2,Census Tract 1013,2161,35,0.016196,0.009560,38.566560,-90.264211
3,Census Tract 1014,1502,95,0.063249,0.009614,38.562243,-90.253522
4,Census Tract 1015,1318,32,0.024279,0.009645,38.548544,-90.264511


In [8]:
north_south_locations.head()

,id,Name,LON,LAT
0,1,Grand Blvd/Fairground Park,-90.217,38.661
1,2,Palm St/Salisbury St,-90.209,38.658
2,3,St Louis Ave,-90.211,38.653
3,4,Cass Ave,-90.213,38.645
4,5,Carr St,-90.213,38.640


In [131]:
k = 12
n = len(full_data)
x0 = []
for i in range(k):
    new_tract = random.randint(0,n-1)
    x0.append(full_data['LON'][new_tract])
    x0.append(full_data['LAT'][new_tract])

In [161]:
lat_bounds = (min(full_data['LAT']),max(full_data['LAT']))
lon_bounds = (min(full_data['LON']),max(full_data['LON']))
bnds = [val for pair in zip([lon_bounds]*12, [lat_bounds]*12) for val in pair]

[(-90.3106306, -90.1886661), (38.5468534, 38.7226495), (-90.3106306, -90.1886661), (38.5468534, 38.7226495), (-90.3106306, -90.1886661), (38.5468534, 38.7226495), (-90.3106306, -90.1886661), (38.5468534, 38.7226495), (-90.3106306, -90.1886661), (38.5468534, 38.7226495), (-90.3106306, -90.1886661), (38.5468534, 38.7226495), (-90.3106306, -90.1886661), (38.5468534, 38.7226495), (-90.3106306, -90.1886661), (38.5468534, 38.7226495), (-90.3106306, -90.1886661), (38.5468534, 38.7226495), (-90.3106306, -90.1886661), (38.5468534, 38.7226495), (-90.3106306, -90.1886661), (38.5468534, 38.7226495), (-90.3106306, -90.1886661), (38.5468534, 38.7226495)]


In [9]:
def distances_to_nearest_stop(tract_locations, metro_locations):
    
    shortest_distance = np.zeros(len(tract_locations))
    
    for i in range(len(tract_locations)):
        cent = np.array([tract_locations['LON'][i],tract_locations['LAT'][i]])
        closest_distance = float('inf') # initialize to infinity
        for j in range(len(metro_locations)):
            metro = np.array(metro_locations[j])
            distance = np.linalg.norm(cent-metro)
            if (distance < closest_distance):
                closest_distance = distance
        shortest_distance[i] = closest_distance
    
    return shortest_distance

In [133]:
def orth_dist(x):
    x = [[x[i], x[i + 1]] for i in range(0, len(x), 2)] # reshape back into 2d array
    model = LinearRegression().fit(np.array(x[0][:]).reshape(-1,1),np.array(x[1][:]))
    b = model.intercept_
    m = model.coef_[0]
    dists = 0
    for stop in x:
        n = abs(-1*m*stop[0]+stop[1]-b)
        d = np.sqrt(m**2+1)
        dists = dists + n/d
    return dists

In [158]:
def fun(new_stops):
    #print(f'shape of new stops: {np.shape(new_stops)}')
    weights = full_data['income weight']
    dist_to_stops = np.dot(weights, distances_to_nearest_stop(full_data, new_stops))
    linearity = orth_dist(new_stops) / 1000
    #print(f'dist to stops: {dist_to_stops:3.3f}, linearity: {linearity:3.3f}')
    return dist_to_stops + linearity

In [159]:
result = minimize(fun, x0, bounds=bnds)
all_centroids = result.x

In [160]:
lp_results = [[all_centroids[i], all_centroids[i + 1]] for i in range(0, len(all_centroids), 2)]

cluster_centroids = pd.DataFrame(data = lp_results, columns = ["long", "lat"])
cluster_centroids['size'] = [8 for i in range(len(cluster_centroids))]

fig = px.scatter_mapbox(cluster_centroids,
                        lat='lat',
                        lon='long',
                        size='size',
                        zoom=10,
                        height=750)
fig.update_layout(mapbox_style="open-street-map")